<a href="https://colab.research.google.com/github/N-aksif-N/Minecolab/blob/free-config/MineColabImproved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@markdown
from IPython.core.display import display, HTML
display(HTML('<p><img style="display: block; margin-left: auto; margin-right: auto;" src="https://github.com/N-aksif-N/MineColab/blob/main/minecolab.png?raw=true" alt="" width="170" height="136" /></p><h1 style="text-align: center;"><span style="color: #00ffff;">MineColab</span></h1><hr /><h2 style="text-align: center;"><span style="color: #99ccff;">Forked By N-Aksif  </span></h2><h2 style="text-align: center;"><span style="color: #FFFFFF;">Run Minecraft Server On Google Colab</span></h2></span></h2><a href="https://github.com/N-aksif-N/MineColab/tree/main" target="_parent"><img align="right" alt="Open In Github"></a>'))

>[🔥 Starting](#scrollTo=Ei7_3ODcUdm_)

>[▶ or 🛑  Console](#scrollTo=JMgdsuPcUuEv)

>[⚓ Options](#scrollTo=HD4lBUT9RcS-)

>[📎  Log](#scrollTo=hfUxmQTgUF8Z)

>[📰  Software](#scrollTo=JkGq3KakW-Bc)

>[🎈  Plugins, mods](#scrollTo=QaNRBRuK8seq)

>[📁 File Management](#scrollTo=F7efc3oOqTVQ)



----

In [ ]:
# @title **[❗]  Set up** {display-mode: "form"}

import subprocess
import re
import glob
import requests
import time
from json import load, dump
import os
try: from jproperties import Properties
except:
  !pip install -q jproperties
  from jproperties import Properties
try: from rich import print
except:
  !pip install -q rich
  from rich import print
if os.path.exists('/content/drive') == False:
  from google.colab import drive, runtime
  drive.mount('/content/drive')
else: from google.colab import runtime
try: from pyngrok import conf, ngrok
except:
  !pip -q install pyngrok
  from pyngrok import conf, ngrok

#------------------------------------------------------------------------------------------------------------------------------------#
def LOG(text):
  if '\n' in str(text):
    print("\n[bold green][ LOG ][/bold green] " + str(text).replace('\n', ''))
  else:
    print("[bold green][ LOG ][/bold green] " + str(text))
def EROR(text):
  print("[bold red][ EROR ][/bold red] " + str(text))
def EXIT():
  runtime.unassign()

def COLABCONFIG(server_name: str):
  return f"/content/drive/My Drive/minecraft/{server_name}/colabconfig.txt"
def COLABCONFIG_LOAD(server_name):
  if os.path.exists(COLABCONFIG(server_name)):
    return load(open(COLABCONFIG(server_name)))
  else:
    colabconfig = {"server_type": "generic"} # using default, if config doesn't exist.
    dump(colabconfig, open("colabconfig.json",'w'))
    EROR('Please checking whether you deleted your colabconfig file or not.')
    EXIT()

SERVERCONFIG = '/content/drive/My Drive/minecraft/server_list.txt'
def SERVER_IN_USE():
  serverconfig = load(open(SERVERCONFIG))
  return serverconfig['server_in_use']

def DOWNLOAD_FILE(url, path, file_name):
  # Download file into file_name thourgh url
  if os.path.exists(path):
    r = requests.get(url)
    if r.status_code == 200:
      LOG('\nDownloading ' + file_name)
      with open(path + '/' + file_name, 'wb') as f:
            f.write(r.content)
    else: EROR('Error '+ str(r.status_code) + "! Most likely you entered an unsupported version. Try running the code again if you think that shouldn't have happened.")
  else: EROR("Error: File didn't exist.")
def EULA(server_name: str):
  # Accepting Eula
  LOG('\nAccepting Eula.')
  EULA = f'/content/drive/My Drive/minecraft/{server_name}/eula.txt'
  if os.path.exists(EULA):
    with open(EULA, 'w') as f:
      f.seek(0)
      f.write("eula=true")
  else: EROR("Error: File didn't exist.")
#--------------------------------------------------------- INSTALL JAVA --------------------------------------------------------------#
def JAVA_VER():
  # Getting java version
  java_ver = !java -version 2>&1 | awk -F[\"\.] -v OFS=. 'NR==1{print $2}'
  return java_ver
def INSTALL_JAVA(version: str):
  # Adoptopen jdk -- optional
  # !wget -qO - https://adoptopenjdk.jfrog.io/adoptopenjdk/api/gpg/key/public | sudo apt-key add -
  # !sudo add-apt-repository --yes https://adoptopenjdk.jfrog.io/adoptopenjdk/deb/ &>/dev/null || echo "Failed to add repo. Still can be ignored if openjdk gets installed."

  # Java 8 is required to run Minecraft versions 1.12 through 1.17. Java 17 is required to run Minecraft version 1.18 and up.
  if version > "1.17":
    !sudo apt-get install openjdk-17-jre-headless &>/dev/null &&echo "Openjdk17 successfully installed" || echo "Failed to install OpenJdk17."
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
    !update-alternatives --set java /usr/lib/jvm/java-17-openjdk-amd64/bin/java
  else:
    !sudo apt-get install openjdk-8-jre-headless &>/dev/null &&echo "Openjdk8 successfully installed" || echo "Failed to install OpenJdk8."
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    !update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
def JAVA_CHECK():
  # Checking whether it is working or not? Currently, Google colab used java 11 as main java.
  if JAVA_VER()[0] == "1" : LOG("Openjdk8 is working correctly, you are good to go.")
  elif JAVA_VER()[0] == '17': LOG("Openjdk17 is working correctly, you are good to go.")
  else:
    EROR("Openjdk doesn't seem to be installed or isn't working, falling back to java" + str(JAVA_VER()[0]))
    EROR('You might experience reduced performance. Minecraft 1.18 and above might fail to launch.')
#------------------------------------------------------- INSTALL SERVER.JAR -------------------------------------------------------------#
# Set up public dict => get download url
SERVERSJAR_URL = {
    'vanilla'   : '/vanilla/vanilla',    'snapshot'  : '/vanilla/snapshot',
    'paper'     : "/servers/paper/",     "purpur"    : "/servers/purpur/",
    "fabric"    : "/modded/fabric",      "forge"     : "/modded/forge",
    "bungeecord": "/proxies/bungeecord", "waterfall" : "/proxies/waterfall/", "velocity"  : "/proxies/velocity"
    }
# The jar file name
def JAR_LIST_RUN(colabconfig: dict):
  return {'vanilla':'server.jar','snapshot': 'server.jar',
          'purpur' : 'server.jar', 'paper': 'server.jar',
          'fabric': 'fabric-launcher.jar', 'forge': 'minecraft_server.' + colabconfig['server_version'] + '.jar',
          'bungeecord' : 'server.jar', 'waterfall' : 'server.jar', 'velocity' : 'server.jar', 'generic': 'server.jar'}
# The jar file install
FORGE_INSTALLER = 'forge': 'forge-installer.jar'
def SERVERSJAR(all = False, jar = False, server_type= '', version = None):
  # Return the download url (jar) or return the detail url (all)
  if all:
    return 'https://serverjars.com/api/fetchAll' + SERVERSJAR_URL[server_type]
  elif jar == True and version != None:
    return 'https://serverjars.com/api/fetchJar' + SERVERSJAR_URL[server_type] + f'/{version}'
  else: EROR('Type error: Wrong given arguments')
def GET_VERSION(server_type):
  # Print available version => get version (string variable)
  LOG('\n Aivailable version:')
  a = requests.get(SERVERSJAR(all = True, server_type= server_type)).json()
  server_version = [hit['version'] for hit in a['response']]
  LOG(server_version)
  version = input('Server version: ')
  while version == '':
    EROR("Invalid versions")
    LOG('\n Aivailable version:', server_version)
    version = input('Server version: ')
  return version
def GET_SERVERJAR(server_name, type_, version, jarname = 'server.jar'):
  # Get jar file
  DOWNLOAD_FILE(url= SERVERSJAR(jar = True, server_type = type_, version= version),
                path = f"/content/drive/My Drive/minecraft/{server_name}",
                file_name= jarname)
  # Forge need to open and run it => Using command instead.
  if type_ == 'forge':
    INSTALL_JAVA(version)
    JAVA_CHECK()
    !java -jar forge-installer.jar --installServer
#-------------------------------------------------------- Java arguments ------------------------------------------------------------------#
def ARGS(type_):
  # Get all the improving java arguments
  AIKAR_FLAGS = '-XX:+UseG1GC -XX:+ParallelRefProcEnabled -XX:MaxGCPauseMillis=200 -XX:+UnlockExperimentalVMOptions -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -XX:G1NewSizePercent=30 -XX:G1MaxNewSizePercent=40 -XX:G1HeapRegionSize=8M -XX:G1ReservePercent=20 -XX:G1HeapWastePercent=5 -XX:G1MixedGCCountTarget=4 -XX:InitiatingHeapOccupancyPercent=15 -XX:G1MixedGCLiveThresholdPercent=90 -XX:G1RSetUpdatingPauseTimePercent=5 -XX:SurvivorRatio=32 -XX:+PerfDisableSharedMem -XX:MaxTenuringThreshold=1 -Dusing.aikars.flags=https://mcflags.emc.gs -Daikars.new.flags=true'
  GC_LOG = '-d64 -Xloggc:gc.log -verbose:gc -XX:+PrintGCDetails -XX:+PrintGCDateStamps -XX:+PrintGCTimeStamps -XX:+UseGCLogFileRotation -XX:NumberOfGCLogFiles=5 -XX:GCLogFileSize=1M'
  GC_LOG2 = '-d64 -Xlog:gc*:logs/gc.log:time,uptime:filecount=5,filesize=1M'
  args = "-Xms10G -Xmx10G"
  if type_ == "paper" or type_ == 'purpur':
    # Improving paper cilent (purpur is an alternative).
    # For more detailed: https://docs.papermc.io/paper/aikars-flags
    args += AIKAR_FLAGS
  # GC_LOGGING
  if JAVA_VER()[0] == "1" : args += GC_LOG
  #elif JAVA_VER()[0] >= '11': args += GC_LOG2  #GC_Logging may not worked in java 11 and above
  return args
#------------------------------------------------------ TUNNEL - COMMAND -------------------- ---------------------------------------#
def CONNECT_NGROK(server_name, port, type_ , colabconfig_proxy):
  # Get colabconfig["proxy"] : dict includes (authtoken, region)
  token = colabconfig_proxy['authtoken']
  !ngrok authtoken $token
  region = colabconfig_proxy['region']
  conf.get_default().region = region
  url = ngrok.connect (port, type_)
  if type_ == 'tcp':
    return 'Your server address is ' + ((str(url).split('"')[1::2])[0]).replace('tcp://', '')
  else:
    return url
def EXIT_NGROK():
  tunnels = ngrok.get_tunnels()
  for tunnel in tunnels:
    ngrok.disconnect(tunnel.public_url)
  ngrok.kill()
  LOG("Ngrok closed.")

def PLAYIT_INSTALL():
  # Download playit
  ! curl -SsL https://playit-cloud.github.io/ppa/key.gpg | gpg --dearmor | sudo tee /etc/apt/trusted.gpg.d/playit.gpg >/dev/null
  ! echo "deb [signed-by=/etc/apt/trusted.gpg.d/playit.gpg] https://playit-cloud.github.io/ppa/data ./" | sudo tee /etc/apt/sources.list.d/playit-cloud.list
  ! sudo apt -qq update && sudo apt install playit &>/dev/null && echo "Playit.gg installed" || echo "Failed to install playit"
def EXECUTE_ARGO():
  # Download argo
  !curl https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 > cloudflared-linux-amd64
  !chmod +x cloudflared-linux-amd64
def RUNCOMMAND(server_name, colabconfig, tunnel_service= '', hide = False):
  args = ARGS(colabconfig["server_type"])
  jar_name = JAR_LIST_RUN(colabconfig)[colabconfig["server_type"]]

  if hide == True:
      # Install needed file: eula.txt and so on.
      !java -server $args -jar $jar_name nogui &>/dev/null &

  else:
    # Checking tunnel service is set or not. If not => get the default in colabconfig['proxy']
    if tunnel_service == '': tunnel_service = colabconfig['proxy']['turnel_service']

    # Starting tunneling and run java file.
    LOG(f'Starting server ({tunnel_service})...')
    if tunnel_service == "ngrok":
      LOG(CONNECT_NGROK(server_name, type_= 'tcp', port= 25565, colabconfig_proxy = colabconfig['proxy']))
      !java -server $args -jar $jar_name nogui
    elif tunnel_service == "playit":
      PLAYIT_INSTALL()
      ! playit & java -server $args -jar $jar_name nogui
    else:
      EXECUTE_ARGO()
      !./cloudflared-linux-amd64 tunnel --url tcp://127.0.0.1:25565 && echo 'Your free tunnel has started!'
      !java -server $args -jar $j

#----------------------------------------------------------- MAIN -------------------------------------------------------------#

def Install_server(proxy , server_name, type_, version = ''):
  if os.path.exists(f'/content/drive/MyDrive/minecraft/{server_name}') == False:
    ! mkdir -p "/content/drive/MyDrive/minecraft/{server_name}"
    if version == '': version = GET_VERSION(server_type)
    elif version == 'vanilla - latest_version':
      vanilla = requests.get(SERVERSJAR(all = True, server_type= type_, version = version)).json()
      version = str(vanilla['response'][0]['version'])
    serverconfig = load(open(SERVERCONFIG))
    serverconfig['server_list'] += [server_name]
    serverconfig['server_in_use'] = server_name
    dump(serverconfig, open(SERVERCONFIG, 'w'))
    LOG("\n[Note: This installation takes approximately 10-15 minutes, wait for 'Completed' after executing the first step.]")
    colabconfig = {"server_type": type_, "server_version": version, "proxy": proxy}
    if type_ == 'forge': jarname = FORGE_INSTALLER
    else: jarname = JAR_LIST_RUN(colabconfig)[type_]
    GET_SERVERJAR(server_name, type_, version, jarname = jarname)
    dump(colabconfig, open(COLABCONFIG(server_name),'w'))
    RUNCOMMAND(server_name, colabconfig, tunnel_service = proxy, hide = True)
    LOG('\nCompleted!')
  else:
    EROR(' You have already installed this server file')
    EXIT()

def Delete_server(server_name):
  if server_name == '':
    server_name = SERVER_IN_USE()
  if os.path.exists(f'/content/drive/MyDrive/minecraft/{server_name}'):
    LOG('Deleting files...')
    serverconfig = load(open(SERVERCONFIG))
    if server_name == '': server_name = serverconfig['server_in_use']
    subprocess.run(['rm', '-rf', f"/content/drive/My Drive/minecraft/{serverconfig['server_in_use']}"])
    serverconfig['server_list'].remove(server_name)
    if serverconfig['server_in_use'] == server_name:
      try: serverconfig['server_in_use']= serverconfig['server_list'][0]
      except: serverconfig['server_in_use'] = ''
    dump(serverconfig, open(SERVERCONFIG, 'w'))
    LOG('Completed')
  else:
    EROR("You haven't installed yet.")
    EXIT()

def Run_server(server_name):
  if server_name == '':
    server_name = SERVER_IN_USE()
  if os.path.exists(f'/content/drive/MyDrive/minecraft/{server_name}'):
    LOG('Server:  ' + server_name)
    %cd "/content/drive/My Drive/minecraft/{server_name}"
    colabconfig = COLABCONFIG_LOAD(server_name)
    INSTALL_JAVA(colabconfig['server_version'])
    JAVA_CHECK()
    LOG("\nFiles:")
    !ls
    EULA(server_name)
    RUNCOMMAND(server_name, colabconfig)
    LOG("Finalized server.")
    EXIT_NGROK()
  else:
    EROR("Please make a minecraft server.")
    EXIT()

#---------------------------------------------------------- Ulitites --------------------------------------------------------------------------#
def Choose_server():
  serverconfig = load(open(SERVERCONFIG))
  if serverconfig['server_list'] == []: LOG(' Creating your minecraft server before choosing')
  else:
    LOG('\nAivailable server:\n')
    for i in range(len(serverconfig['server_list'])):
      LOG(serverconfig['server_list'][i])
    serverconfig['server_in_use'] = input('\nYour minecraft server: ')
    dump(serverconfig, open(SERVERCONFIG, 'w'))
    LOG('Completed')

def Custom_props(server_name, server_icon, motd):
  if server_name == '': server_name = SERVER_IN_USE()
  if server_icon != '':
    DOWNLOAD_FILE(
        url = server_icon,
        path = f'content/drive/MyDrive/minecraft/{server_name}',
        file_name = 'servericon.png'
    )
  if motd != '':
    server_properties = Properties()
    with open(f"/content/drive/My Drive/minecraft/{server_name}/server.properties", "rb") as f:
        server_properties.load(f, "utf-8")
    server_properties["motd"] = motd
    with open(f"/content/drive/My Drive/minecraft/{server_name}/server.properties", "wb") as f:
        server_properties.store(f, encoding="utf-8")
  LOG('Completed')

def Serverproperties(server_name, Slots, Gamemode, Difficulty, Cracked, PVP, Command_block, Fly, Animals,
                     Monsters, Villagers, Nether, Force_gamemode, Spawn_protection):
  if server_name == '':
    server_name = SERVER_IN_USE()
  if server_name != '':
    LOG("Configuring")
    server_properties = Properties()
    with open(f"/content/drive/My Drive/minecraft/{server_name}/server.properties", "rb") as f:
        server_properties.load(f, "utf-8")
    server_properties["max-players"] = str(Slots)
    server_properties["gamemode"] = Gamemode
    server_properties["difficulty"] = Difficulty
    server_properties["online-mode"] = str(Cracked).lower()
    dict_ = {'pvp': PVP, 'enable-command-block': Command_block, 'allow-flight': Fly, 'spawn-animals': Animals, 'spawn-monsters': Monsters,
            'spawn-npcs': Villagers, 'allow-nether': Nether, 'force-gamemode': Force_gamemode, 'spawn-protection': Spawn_protection}
    for keys, value in dict.items():
      server_properties[keys] = str(value).lower()
    with open(f"/content/drive/My Drive/minecraft/{server_name}/server.properties", "wb") as f:
        server_properties.store(f, encoding="utf-8")
    LOG('Completed')
  else:
    EROR("Have you created your server file?")
    EXIT()

def Software(server_name, server_type, version = ''):
  if server_name == '':
    server_name = SERVER_IN_USE()
  if server_name != '':
    colabconfig = COLABCONFIG_LOAD(server_name)
    Delete_server(server_name)
    Install_server(colabconfig['proxy'] , server_name, server_type)
  else:
    EROR("You haven't isntall yet!")
    EXIT()

def Latest_log():
  server_folder = SERVER_IN_USE()
  if os.path.exists(f'/content/drive/My Drive/minecraft/{server_folder}/logs/latest.log'):
    with open(f'/content/drive/My Drive/minecraft/{server_folder}/logs/latest.log', 'r') as f:
      print(f.read())
  else: EROR("Error: File didn't exists")

#--------------------------------------------------- SERVER - PLUGINS/MODS/MODPACKS SEARCH ----------------------------------------------------------#
def facetsInURL(colabconfig, software, categories, versions, project_types, proj_types, index):
  if software == 'Modrinth':
    if categories == 'none': cats = ""
    else: cats = f'["categories:{[categories]}"]';
    if versions == 'none': vers = ""
    else: vers = f'["versions:{[versions]}"]';
    if project != 'none': proj_types = f'["project_type:{[proj_types]}"]'
    else: proj_types = ''
    facets = "["
    if cats != "": facets += cats
    if vers != "":
      if cats != "": facets += ","
      facets += vers
    if proj_types != "":
      if cats != "" or vers != "": facets += ","
      facets += proj_types
    facets += "]"; facetsInURL = "";
    if facets != "[]": facetsInURL = f'&facets={facets}&index={index}'
  else:
    if categories != 'none':
      if categories == 'fabric': categories = 4 #Fabric
      elif categories == 'forge': categories = 1 #Forge
      elif categories == 'quilt': categories = 5 # quilt
    else: categories = 0 #Any
    if index == "relevance": index = 1 # Featured
    elif index == "downloads": index = 6 #TotalDownloads
    elif index == "follows": index = 2 #Popularity
    elif index == "newest": index = 11 #ReleasedDate
    elif index == "updated": index = 3 #LastUpdated
    else: index = 2
    if versions != "none": versions = f"&gameVersion={versions}"
    else: versions = ""
    facetsInURL = f"&modLoaderType={categories}" + versions + f"&sortField={index}"
  return facetsInURL
def SEARCH(software, server_name, search_name, categories, versions, project_types, proj_types, index):
  colabconfig = COLABCONFIG_LOAD(server_name)
  project = {}

  LOG(f'\nSearching for the related of {search_name} ...')
  if software == "Modrinth":
    facetsInURL = facetsInURL(colabconfig, software, categories, versions, project_types, proj_types, index)
    rJSON = requests.get(f'https://api.modrinth.com/v2/search?query={search_name}{facetsInURL}').json()
    # Get the list of all relevant project
    for hit in rJSON['hits']:
      if hit['server_side'] == 'optional' or hit['server_side'] == 'required':
        LOG(hit['slug'], " : ", hit['description'], ' : ', hit["versions"])
        project[hit['slug']] = hit['project_id']
  elif software == 'Curseforge':
    facetsInURL = facetsInURL(colabconfig, categories, versions, project_types, proj_types)
    # Because I haven't found any corresponded of project types in the search engine of Curseforge, I don't use it for searching.
    # The gameid of Minecraft is 432
    rJSON = requests.get(f"https://api.curse.tools/v1/cf/mods/search?gameId=432&searchFilter={search_name}{facetsInURL}").json()
    # Get the list of all relevant project
    for hit in rJSON["data"]:
      print(f"Curseforge doesn't have the exact searching key for cilent-side or server-side => you may get errors when running this {proj_types}")
      gameversions = [vers["gameVersion"] for vers in hit["sortableGameVersions"]]
      print(hit["name"], ' : ', hit["summary"], ' : ', gameversions)
      project[hit['name']] = str(hit["id"])
  if project == {}:
    EROR(f"\nSomething went wrong. Please check your search name.")
    EXIT()
  else:
    LOG('\nType the project_name you want to download')
    project_names= input('Project_name: ')
    while project_names not in project:
      LOG('\nWrong project_names please type aigain. If you want to quit, type "None".')
      project_names= input('\nProject_name: ')
      if project_names == 'None': break
  return [project, project_names]
#--------------------------------------------------- Ulities ----------------------------------------------------------#
def Install_(server_name, software, url, search_name, categories, versions, project_types, index):
  if server_name == '':
    server_name = SERVER_IN_USE()
  if url == '':
    if versions == 'default': versions = colabconfig["server_version"]
    if project_types == 'default' :
      if 'fabric' in categories or 'forge' in categories : proj_types = 'mods'
      elif 'paper' in categories or 'purpur' in categories : proj_types = 'plugins'
    if categories == 'default': categories = colabconfig['server_type']
    a = SEARCH(software, server_name, search_name, categories, versions, project_types, proj_types, index)
    project = a[0]; project_names = a[1]; check = False
    if project_names != 'None':
        project_id = project[project_names]
        if software == 'Modrinth': rJSON = requests.get(f'https://api.modrinth.com/v2/project/{project_id}/version').json()
        else: rJSON = requests.get(f'https://api.curse.tools/v1/cf/mods/{project_id}').json()['latestFilesIndexes']
        for data in rJSON:
          if software == 'Curseforge': data = data["gameVersion"]
          else: data = data['game_versions']
          for game_version in data:
            if versions == game_version:
              if software == 'Modrinth': files = data['files'][0]; url = files['url']
              else: files = data['fileId']; url = f'https://www.curseforge.com/api/v1/mods/{project_id}/files/{fileId}/download'
              DOWNLOAD_FILE(url= url,
                            path = f'/content/drive/My Drive/minecraft/{server_name}/{proj_types}',
                            file_name= files['filename'])
              check = True
          if check == True: print('\nCompleted.'); break
        if check == False: EROR(f"\nCan't find the exact file. It seems that {software} doesn't support {versions[0]} version for {project_names}")
    else: LOG('Stopping...')
  else:
    filename = url[url.find("/") + 1:]
    while filename.find("/")!= -1:
      filename = filename[filename.find("/") + 1:]
    DOWNLOAD_FILE(url= url, path = f'/content/drive/My Drive/minecraft/{server_name}/{proj_types}', file_name= filename)

def Backup_files(file_name = '', file_backup = '', server_name= '', server_backup = '', path = '/content/drive/MyDrive/minecraft'):
  # Get the default server
  if server_name == '':
    server_name = SERVER_IN_USE()
  # Settings path
  if file_name != '':
    path1 = path + f'/{server_name}/' + file_name
    if server_backup != '':
      if os.path.exists(f'/content/drive/MyDrive/minecraft/{server_backup}') == False:
        !mkdir '/content/drive/MyDrive/minecraft/{server_backup}'
    if file_backup != '' and server_backup != '': path2 = path + f'/{server_backup}/' + file_backup
    elif file_backup != '' and server_backup == '': path2 = path + f'/{server_name}/' + file_backup
    elif file_backup == '' and server_backup != '':  path2 = path + f'/{server_backup}/' + '(back-up)'
    else: path2 = path + f'/{server_name}/' + '(back-up)'
  else:
    path1 = path + f'/{server_name}'
    if server_backup != '': path2 = path + f'/{server_backup}'
    else: path2 = path + f'/{server_name}' + '(back-up)'
  # Zipping
  !zip -r "{path2}".zip path1 &>dev/null && echo "Zipping done!" || echo "Zipping faled."


#------------------------------------------------------------------------------------------------------------------------------------#
!command -v filebrowser &>/dev/null || curl -fsSL https://raw.githubusercontent.com/filebrowser/get/master/get.sh | bash &>/dev/null
!sudo apt update &>/dev/null &&echo 'apt cache successfully updated' || echo "apt cache update failed, you might receive stale packages"
!sudo apt upgrade &>/dev/null &&echo 'apt cache successfully upgraded' || echo "apt cache upgrade failed, you might receive stale packages"
if os.path.exists('/content/drive/MyDrive/minecraft') == False: subprocess.run(['mkdir', '-p', '/content/drive/MyDrive/minecraft'])

if os.path.exists(SERVERCONFIG) == False:
  with open(SERVERCONFIG, 'w') as f:
    dump({"server_list": [], "server_in_use": ""}, f)

LOG('Completed')

---

# 🔥 **Starting**
---


In [ ]:
# @markdown ####**Choose your server file name**
server_name = "minecraft" # @param {type:"string"}
# @markdown ####You can either use [ngrok](https://ngrok.com/), [playit.gg](https://playit.gg/) or [cloudflare's argo](https://www.cloudflare.com/). Ngrok is easy to setup and doesn't requires anything to be installed by the clients but it can often be quite unreliable. Argo doesn't have such limitations but requires a bit more work. Playit has provided limited versions but with a static domain.
# Standard turnel service ( Aivailable turnel: ngrok, argo, playit)
turnel_service = 'ngrok' #@param ['ngrok', 'argo', 'playit']
if turnel_service == 'ngrok':
  authtoken = ''
  region = ''
  LOG('Get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken')
  authtoken = input('Your authtoken: ') # '2b4aAThNSdouorMoFZTy55kGHLZ_5AUszgVXmfh8f45kSLr2N'
  LOG('Available Regions: ')
  LOG(' ap - Asia/Pacific (Singapore)')
  LOG(' au - Australia (Sydney)')
  LOG(' eu - Europa (Frankfurt - Germany)')
  LOG(' in - India (Mumbai)')
  LOG(' jp - Japan (Tokyo)')
  LOG(' sa - America (São Paulo - Brazil)')
  LOG(' us - United States (Ohio)')
  region = input('Region: ')
Install_server(
  #------------------------------#
    proxy = {
        "turnel_service" : turnel_service,
        'authtoken'   : authtoken,
        "region"  : region
    },
  #------------------------------#
    server_name = server_name, # your server name

    type_ = "vanilla", # server type
    version = 'vanilla - latest_version'
  #------------------------------#
)

In [ ]:
# @markdown ####**Choosing minecraft server**

Choose_server()

In [ ]:
# @markdown ####**Delete server**

Delete_server(
    #------------------------------#
    # @markdown Leaving emty string   -> set to default files
    server_name = '' # @param {type: "string"}
    #------------------------------#
)

-----------------------------------------------------

# ▶ **or** 🛑  **Console**
---
The main console for your minecraft server

- **[Ngrok](https://ngrok.com)**
  + Follow the prompts.
  + The IP will change whenever you restart the server.
- **[Cloudflare's argo](https://www.cloudflare.com/)** :
    - If the 'Your free tunnel has started!' notification appears => Done.
    - Access to your server:
    1. Download [Cloudflared client](https://github.com/cloudflare/cloudflared/releases/).
    2. Launch the binary with `<your Cloudflare file name> access tcp --hostname <tunnel_address> --url 127.0.0.1:25565` (note: tunnel_address is your address which has been set on your Cloudflare)
    4. Finally, connect to `127.0.0.1:25565` from the minecraft client which is located in that machine.
- **[PlayIt](https://playit.gg/)**: follow the prompts.

In [ ]:
Run_server(
    #------------------------------#
    # @markdown ####Leaving emty string   -> set to default files
    server_name = '' # @param {type: 'string'}
    #------------------------------#
)

---

# ⚓ **Options**
---


In [ ]:
# @markdown ###**Server_custom**

# @markdown ####Server icon: **upload image** and get **direct link** from [[ Image ]](https://postimages.org/)
Server_icon = '' # @param {type: 'string'}
# @markdown ####[[ MOTD create **tool** ]](https://mctools.org/motd-creator)
MOTD = '' # @param {type: 'string'}


Custom_props(
    #------------------------------#
    server_name = '', # Customized server file
    #------------------------------#
    server_icon = Server_icon,
    motd = MOTD
    #------------------------------#

)

In [ ]:
# @markdown ###**Server properties**

Slots = 20 # @param {type:"slider", min:0, max:100, step:1}
Gamemode = "survival" # @param ["survival", "creative", "adventure", "spectator"]
Difficulty = "easy" # @param ["peaceful", "easy", "normal", "hard"]
Cracked = True # @param {type:"boolean"}
PVP = True # @param {type:"boolean"}
Command_block = True # @param {type:"boolean"}
Fly = True # @param {type:"boolean"}
Animals = True # @param {type:"boolean"}
Monsters = True # @param {type:"boolean"}
Villagers = True # @param {type:"boolean"}
Nether = True # @param {type:"boolean"}
Force_gamemode = False # @param {type:"boolean"}
Spawn_protection = 16 # @param {type:"slider", min:0, max:100, step:1}



Serverproperties(
    server_name = '',

    Slots,
    Gamemode,
    Difficulty,
    Cracked,
    PVP,
    Command_block,
    Fly,
    Animals,
    Monsters,
    Villagers,
    Nether,
    Force_gamemode,
    Spawn_protection

)

---

# 📎  **Log**
---


In [ ]:
# @markdown **Shows latest log of your minecraft server**
# Getting server_folder

Latest_log()

---


# 📰  **Software**
---
Change the software (vanilla) to others platforms

---


+ Please checking whether your minecraft server is in Gdrive or not.


In [ ]:
Software(

    # Default server
    server_name = '',

    # Server_version
    version = '', # Customizable version

    # @markdown ####*Choosing server type*
    server_type = "fabric" # @param ["vanilla", "snapshot", "paper", "purpur", "forge", "fabric", 'bungeecord', 'waterfall', 'velocity']

)

---

# 🎈  **Plugins, mods**
---


In [ ]:
# @title **Download modpack/mod/plugin from [curseforge](https://www.curseforge.com/Minecraft) and [modrinth](https://modrinth.com/)**
#@markdown ### **Your search name**

search_name = '' # @param {type: 'string'}

# @markdown ####Choose the place to download mod/modpack/plugin
software = 'Curseforge' # @param ["Curseforge", "Modrinth"]
url = '' # Customized urlmodpack (just from curseforge or modpack)

# @markdown ###**Details:**
# @markdown
# @markdown #### (none -> don't search, default -> set up according to colabconfig file)
categories = 'none'   #@param ['none', 'default', 'vanilla', 'fabric', 'forge', 'paper', 'purpur', 'bungeecord', 'velocity', 'waterfall']
versions = "default" # @param ["none", "default"] {allow-input: true}
project_types = 'none' # @param ['none', 'default', 'mod', 'plugin']
index = 'relevance' #@param ['relevance', 'downloads', 'follows', 'newest', 'updated']

Install_(

    server_name = '', #Server_name -- left => set to default
    software, url,

    search_name,
    categories, versions, project_types, index
)

---

# **📁 File Management**
---


In [ ]:
# @markdown Back up server or file?
choice = 'server' # @param ['server', 'file']
file_name = ''; file_backup = ''; server_name= ''; server_backup = ''
if choice == 'server':
  server_name = input('Server name: ')
  server_backup = input('Back up server name: ')
else:
  server_name = input('Server name: ')
  file_name = input('File name')
  file_backup = input('File back up name: ')
Backup_files(
    #------------------------------#
    file_name = file_name,
    file_backup = file_backup,
    server_name= server_name,
    server_backup = server_backup,
    path = '/content/drive/MyDrive/minecraft'
    #------------------------------#
)